# Import Dependencies and data 
#### Read the data using pandas 

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
from sklearn import tree

In [ ]:
# Study data files
data1 = "../data/nba odds 2014-15.csv"
data2 = "../data/nba odds 2015-16.csv"
data3 = "../data/nba odds 2016-17.csv"
data4 = "../data/nba odds 2017-18.csv"

# Read the mouse data and the study results
data1 = pd.read_csv(data1)
data2 = pd.read_csv(data2)
data3 = pd.read_csv(data3)
data4 = pd.read_csv(data4)

# Merge all the different seasons 

In [ ]:
def set_win(row):
    if row['Point Dif'] > 0:
        return 1
    else:
        return 0
    
def set_spread_win(row):
    handi = row['Point Dif'] + row['Spread']
    if handi > 0:
        return 1
    else:
        return 0
    
def set_ou_win(row):
    if row['Game Total'] > row['Over']:
        return 1
    else:
        return 0

def data_clean(df):
    df = df.replace({'Team': {'Atlanta':'ATL','Boston':'BOS','Brooklyn':'BRK',
                              'Charlotte':'CHO','Chicago':'CHI','Cleveland':'CLE',
                              'Dallas':'DAL','Denver':'DEN','Detroit':'DET',
                              'GoldenState':'GSW','Houston':'HOU','Indiana':'IND',
                              'LAClippers':'LAC','LALakers':'LAL','Memphis':'MEM',
                              'Miami':'MIA','Milwaukee':'MIL','Minnesota':'MIN',
                              'NewOrleans':'NOP','NewYork':'NYK','OklahomaCity':'OKC',
                              'Orlando':'ORL','Philadelphia':'PHI','Phoenix':'PHO',
                              'Portland':'POR','Sacramento':'SAC','SanAntonio':'SAS',
                              'Toronto':'TOR','Utah':'UTA','Washington':'WAS'
                    }})

    p_dif = []
    p_total = []
    spread = []
    over = []

    # Iterate through table and calculate point dif and winners and insert into list 
    for i in range(1, df.shape[0], 2):
        home_dif = df['Final'].iloc[i] - df['Final'].iloc[i-1]
        away_dif = df['Final'].iloc[i-1] - df['Final'].iloc[i]
        total = df['Final'].iloc[i] + df['Final'].iloc[i-1]
        if df['Close'].iloc[i] == 'pk':
            spread.append(0)
            spread.append(0)
            over.append(float(df['Close'].iloc[i-1]))
            over.append(float(df['Close'].iloc[i-1]))
        elif df['Close'].iloc[i-1] == 'pk':
            spread.append(0)
            spread.append(0)
            over.append(float(df['Close'].iloc[i]))
            over.append(float(df['Close'].iloc[i]))
        elif df['ML'].iloc[i] < df['ML'].iloc[i-1]:
            spread.append(float(df['Close'].iloc[i]))
            spread.append(-float(df['Close'].iloc[i]))
            over.append(float(df['Close'].iloc[i-1]))
            over.append(float(df['Close'].iloc[i-1]))
        elif df['ML'].iloc[i] > df['ML'].iloc[i-1]:
            spread.append(-float(df['Close'].iloc[i-1]))
            spread.append(float(df['Close'].iloc[i-1]))
            over.append(float(df['Close'].iloc[i]))
            over.append(float(df['Close'].iloc[i]))
        else:
            spread.append(0)
            spread.append(0)
            over.append(float(df['Close'].iloc[i-1]))
            over.append(float(df['Close'].iloc[i-1]))

        p_total.append(total)
        p_total.append(total)
        p_dif.append(away_dif)
        p_dif.append(home_dif)
            
    # Add point dif column and data         
    df['Point Dif'] = p_dif
    df['Game Total'] = p_total
    df['Spread'] = spread
    df['Over'] = over
    df = df.reset_index()
    
    # add the result column 
    df = df.assign(Win=df.apply(set_win, axis=1))
    df = df.assign(Win_Spread=df.apply(set_spread_win, axis=1)) 
    df = df.assign(Win_Over=df.apply(set_ou_win, axis=1)) 
    
    point_dict = {}
    net_points = []
    wd = {}
    w = []    

    for i in df.iterrows():
    #point difference
        if i[1]['Team'] in point_dict:
            net_points.append(point_dict[i[1]['Team']])
            point_dict[i[1]['Team']] = point_dict[i[1]['Team']] + int(i[1]['Point Dif'])
            w.append(wd[i[1]['Team']])
            wd[i[1]['Team']] = wd[i[1]['Team']] + int(i[1]['Win'])
        else:
            point_dict[i[1]['Team']] = int(i[1]['Point Dif'])
            net_points.append(0)
            wd[i[1]['Team']] = int(i[1]['Win'])
            w.append(0)
    
    df['season_wins'] = w        
    df['net_points'] = net_points
    df = df.sort_values(by=['Team','season_wins','Date'])
    return df

In [ ]:
clean1 = data_clean(data1)
clean2 = data_clean(data2)
clean3 = data_clean(data3)
clean4 = data_clean(data4)

frames = [clean1,clean2,clean3,clean4]
    
df = pd.concat(frames)
df.drop(df.columns[23:],axis=1,inplace=True)
df

In [ ]:
df.to_csv('historical_odds.csv')

In [ ]:
odds_df = pd.read_csv('historical_odds.csv')
odds_df

# Cleaning Up the Dataframe

In [ ]:
cols = [0,1,3,6,7,8,9,10,11,12,14,15,16]
odds_df.drop(odds_df.columns[cols],axis=1,inplace=True)
odds_df

In [ ]:
odds_df = odds_df.replace({'VH': {'V': 0, 'H': 1, 'N':0}})
odds_df = odds_df.rename(columns={"VH": "Home"})
odds_df = odds_df.dropna()

odds_df

In [ ]:
odds_df.to_csv('historical_odds.csv')

# Manually add dates

In [2]:
odds_df = pd.read_csv('historical_odds.csv')
cols = [0]
odds_df.drop(odds_df.columns[cols],axis=1,inplace=True)
odds_df

,Date,Home,Team,ML,Spread,Over,Win,Win_Spread,Win_Over,season_wins,net_points
0,2014-10-29,0,ATL,161,4.5,198.5,0,0,1,0,0
1,2014-11-01,1,ATL,-650,-11.0,193.5,1,0,1,0,-7
2,2014-11-05,0,ATL,315,8.0,202.0,0,1,0,1,3
3,2014-11-07,0,ATL,105,1.5,194.5,0,0,1,1,1
4,2014-11-08,1,ATL,-290,-6.5,194.5,1,1,1,1,-2
...,...,...,...,...,...,...,...,...,...,...,...
9835,2018-04-03,0,WAS,325,8.5,216.5,0,0,1,42,71
9836,2018-04-05,0,WAS,210,6.0,220.5,0,1,1,42,55
9837,2018-04-06,1,WAS,-850,-10.5,210.0,0,0,0,42,51
9838,2018-04-10,1,WAS,-350,-7.5,204.5,1,1,1,42,45


In [3]:
stats_df = pd.read_csv('../data/nba.games.stats.csv')
raw_df = pd.merge(odds_df,stats_df,on=['Team','Date'])
raw_df

,Date,Home_x,Team,ML,Spread,Over,Win,Win_Spread,Win_Over,season_wins,...,Opp.FreeThrows,Opp.FreeThrowsAttempted,Opp.FreeThrows.,Opp.OffRebounds,Opp.TotalRebounds,Opp.Assists,Opp.Steals,Opp.Blocks,Opp.Turnovers,Opp.TotalFouls
0,2014-10-29,0,ATL,161,4.5,198.5,0,0,1,0,...,27,33,0.818,16,48,26,13,9,9,22
1,2014-11-01,1,ATL,-650,-11.0,193.5,1,0,1,0,...,18,21,0.857,11,44,25,5,5,18,26
2,2014-11-05,0,ATL,315,8.0,202.0,0,1,0,1,...,27,38,0.711,11,50,25,7,9,19,15
3,2014-11-07,0,ATL,105,1.5,194.5,0,0,1,1,...,20,27,0.741,11,51,31,6,7,19,30
4,2014-11-08,1,ATL,-290,-6.5,194.5,1,1,1,1,...,8,11,0.727,13,44,26,2,6,15,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9560,2018-04-03,0,WAS,325,8.5,216.5,0,0,1,42,...,18,27,0.667,10,46,26,13,3,9,14
9561,2018-04-05,0,WAS,210,6.0,220.5,0,1,1,42,...,22,28,0.786,5,35,26,10,3,16,14
9562,2018-04-06,1,WAS,-850,-10.5,210.0,0,0,0,42,...,16,23,0.696,7,50,24,5,5,18,22
9563,2018-04-10,1,WAS,-350,-7.5,204.5,1,1,1,42,...,22,27,0.815,13,44,22,14,1,16,18


In [4]:
raw_df.to_csv('first_merge.csv')

In [5]:
# team_dict = {}
# avg_for_fg = []
# avg_for_fgp = []
# avg_opp_fgp = []
# avg_for_ft = []
# avg_for_ftp = []
# avg_opp_ft_att = []
# for_3p = []
# avg_for_3p = []
# avg_opp_3p = []
# avg_for_blk = []
# avg_opp_blk = []
# avg_for_to = []
# avg_opp_to = []
# ttl_blk = []

raw_df['avg_for_fg'] = raw_df['FieldGoals'].rolling(2).mean()
raw_df['avg_for_fgp'] = raw_df['FieldGoals.'].rolling(2).mean()
raw_df['avg_opp_fgp'] = raw_df['Opp.FieldGoals.'].rolling(2).mean()
raw_df['avg_for_3p'] = raw_df['X3PointShots'].rolling(2).mean()
raw_df['avg_for_3pp'] = raw_df['X3PointShots.'].rolling(2).mean()
raw_df['avg_opp_3pp'] = raw_df['Opp.3PointShots.'].rolling(2).mean()
raw_df['avg_for_ft'] = raw_df['FreeThrows'].rolling(2).mean()
raw_df['avg_for_ftp'] = raw_df['FreeThrows.'].rolling(2).mean()
raw_df['avg_opp_fta'] = raw_df['Opp.FreeThrowsAttempted'].rolling(2).mean()
raw_df['avg_for_rb'] = raw_df['TotalRebounds'].rolling(2).mean()
raw_df['avg_opp_rb'] = raw_df['Opp.TotalRebounds'].rolling(2).mean()
raw_df['avg_for_blk'] = raw_df['Blocks'].rolling(2).mean()
raw_df['avg_opp_blk'] = raw_df['Opp.Blocks'].rolling(2).mean()
raw_df['avg_for_to'] = raw_df['Turnovers'].rolling(2).mean()
raw_df['avg_opp_to'] = raw_df['Opp.Turnovers'].rolling(2).mean()


# for i in raw_df.iterrows():
# #point difference
#     if i[1]['Team'] in team_dict:
#         for_fg.append(point_dict[i[1]['Team']]['for_fg'])
#         team_dict[i[1]['Team']]['for_fg'] = team_dict[i[1]['Team']]['for_fg'] + int(i[1]['FieldGoals'])
#         for_3p.append(point_dict[i[1]['Team']]['for_3p'])
#         team_dict[i[1]['Team']]['for_3p'] = team_dict[i[1]['Team']]['for_3p'] + int(i[1]['X3PointShots'])
#         ttl_blk.append(point_dict[i[1]['Team']]['ttl_blk'])
#         team_dict[i[1]['Team']]['ttl_blk'] = team_dict[i[1]['Team']]['ttl_blk'] + int(i[1]['Blocks'])
#         ttl_blk.append(point_dict[i[1]['Team']]['ttl_blk'])
#         team_dict[i[1]['Team']]['ttl_blk'] = team_dict[i[1]['Team']]['ttl_blk'] + int(i[1]['Blocks'])
#     else:
#         game_cnt = 1
#         point_dict[i[1]['Team']] = int(i[1]['Point Dif'])
#         net_points.append(0)
#         wd[i[1]['Team']] = int(i[1]['Win'])
#         w.append(0)
        
raw_df

,Date,Home_x,Team,ML,Spread,Over,Win,Win_Spread,Win_Over,season_wins,...,avg_opp_3pp,avg_for_ft,avg_for_ftp,avg_opp_fta,avg_for_rb,avg_opp_rb,avg_for_blk,avg_opp_blk,avg_for_to,avg_opp_to
0,2014-10-29,0,ATL,161,4.5,198.5,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-11-01,1,ATL,-650,-11.0,193.5,1,0,1,0,...,0.3415,17.0,0.6435,27.0,39.5,46.0,7.0,7.0,14.5,13.5
2,2014-11-05,0,ATL,315,8.0,202.0,0,1,0,1,...,0.3345,16.5,0.7425,29.5,37.0,47.0,5.5,7.0,12.5,18.5
3,2014-11-07,0,ATL,105,1.5,194.5,0,0,1,1,...,0.2900,14.0,0.7480,32.5,37.5,50.5,4.0,8.0,16.0,19.0
4,2014-11-08,1,ATL,-290,-6.5,194.5,1,1,1,1,...,0.3335,24.0,0.7735,19.0,39.5,47.5,4.0,6.5,13.5,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9560,2018-04-03,0,WAS,325,8.5,216.5,0,0,1,42,...,0.4385,14.0,0.8480,20.0,36.5,43.0,2.5,2.0,16.5,13.0
9561,2018-04-05,0,WAS,210,6.0,220.5,0,1,1,42,...,0.4145,14.0,0.7765,27.5,38.5,40.5,4.0,3.0,16.0,12.5
9562,2018-04-06,1,WAS,-850,-10.5,210.0,0,0,0,42,...,0.4070,15.0,0.8200,25.5,43.0,42.5,5.5,4.0,13.5,17.0
9563,2018-04-10,1,WAS,-350,-7.5,204.5,1,1,1,42,...,0.3635,17.0,0.7915,25.0,48.0,47.0,3.5,3.0,16.0,17.0


In [6]:
raw_df = raw_df.rename(columns={"Home_x":"home"})

raw_df = raw_df[["Team","home", "ML","Spread","Over","season_wins", "net_points", "avg_for_fg",
              "avg_for_fgp", "avg_opp_fgp","avg_for_3p","avg_for_3pp","avg_opp_3pp","avg_for_ft",
                "avg_for_ftp","avg_opp_fta","avg_for_rb","avg_opp_rb","avg_for_blk","avg_opp_blk",
                 "avg_for_to","avg_opp_to","Win","Win_Spread","Win_Over"]]

In [7]:
raw_df

,Team,home,ML,Spread,Over,season_wins,net_points,avg_for_fg,avg_for_fgp,avg_opp_fgp,...,avg_opp_fta,avg_for_rb,avg_opp_rb,avg_for_blk,avg_opp_blk,avg_for_to,avg_opp_to,Win,Win_Spread,Win_Over
0,ATL,0,161,4.5,198.5,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1
1,ATL,1,-650,-11.0,193.5,0,-7,37.5,0.5035,0.3970,...,27.0,39.5,46.0,7.0,7.0,14.5,13.5,1,0,1
2,ATL,0,315,8.0,202.0,1,3,36.5,0.4600,0.4160,...,29.5,37.0,47.0,5.5,7.0,12.5,18.5,0,1,0
3,ATL,0,105,1.5,194.5,1,1,40.5,0.4375,0.4720,...,32.5,37.5,50.5,4.0,8.0,16.0,19.0,0,0,1
4,ATL,1,-290,-6.5,194.5,1,-2,38.0,0.4345,0.4855,...,19.0,39.5,47.5,4.0,6.5,13.5,17.0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9560,WAS,0,325,8.5,216.5,42,71,37.5,0.4925,0.5180,...,20.0,36.5,43.0,2.5,2.0,16.5,13.0,0,0,1
9561,WAS,0,210,6.0,220.5,42,55,42.5,0.5140,0.5285,...,27.5,38.5,40.5,4.0,3.0,16.0,12.5,0,1,1
9562,WAS,1,-850,-10.5,210.0,42,51,41.0,0.4510,0.5130,...,25.5,43.0,42.5,5.5,4.0,13.5,17.0,0,0,0
9563,WAS,1,-350,-7.5,204.5,42,45,38.0,0.4480,0.4300,...,25.0,48.0,47.0,3.5,3.0,16.0,17.0,1,1,1


In [8]:
final_df = raw_df.dropna()
final_df = final_df.reset_index()
cols = [0]
final_df.drop(final_df.columns[cols],axis=1,inplace=True)
final_df

,Team,home,ML,Spread,Over,season_wins,net_points,avg_for_fg,avg_for_fgp,avg_opp_fgp,...,avg_opp_fta,avg_for_rb,avg_opp_rb,avg_for_blk,avg_opp_blk,avg_for_to,avg_opp_to,Win,Win_Spread,Win_Over
0,ATL,1,-650,-11.0,193.5,0,-7,37.5,0.5035,0.3970,...,27.0,39.5,46.0,7.0,7.0,14.5,13.5,1,0,1
1,ATL,0,315,8.0,202.0,1,3,36.5,0.4600,0.4160,...,29.5,37.0,47.0,5.5,7.0,12.5,18.5,0,1,0
2,ATL,0,105,1.5,194.5,1,1,40.5,0.4375,0.4720,...,32.5,37.5,50.5,4.0,8.0,16.0,19.0,0,0,1
3,ATL,1,-290,-6.5,194.5,1,-2,38.0,0.4345,0.4855,...,19.0,39.5,47.5,4.0,6.5,13.5,17.0,1,1,1
4,ATL,0,-130,-2.0,197.5,2,5,30.0,0.3935,0.4550,...,11.5,39.5,42.0,4.0,4.0,11.5,15.0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9559,WAS,0,325,8.5,216.5,42,71,37.5,0.4925,0.5180,...,20.0,36.5,43.0,2.5,2.0,16.5,13.0,0,0,1
9560,WAS,0,210,6.0,220.5,42,55,42.5,0.5140,0.5285,...,27.5,38.5,40.5,4.0,3.0,16.0,12.5,0,1,1
9561,WAS,1,-850,-10.5,210.0,42,51,41.0,0.4510,0.5130,...,25.5,43.0,42.5,5.5,4.0,13.5,17.0,0,0,0
9562,WAS,1,-350,-7.5,204.5,42,45,38.0,0.4480,0.4300,...,25.0,48.0,47.0,3.5,3.0,16.0,17.0,1,1,1


In [9]:
def ml_pay(row):
    if row['Win'] == 1:
        if row['ML'] < 0:
            pay = 100/(-row['ML'])*100
        else:
            pay = row['ML']
        return pay
    else:
        return -100
    
def s_pay(row):
    if row['Win_Spread'] == 1:
        return 91
    else:
        return -100
    
def o_pay(row):
    if row['Win_Over'] == 1:
        return 91
    else:
        return -100
    
final_df = final_df.assign(ML_Payout=final_df.apply(ml_pay, axis=1))
final_df = final_df.assign(S_Payout=final_df.apply(s_pay, axis=1))
final_df = final_df.assign(O_Payout=final_df.apply(o_pay, axis=1))
final_df

,Team,home,ML,Spread,Over,season_wins,net_points,avg_for_fg,avg_for_fgp,avg_opp_fgp,...,avg_for_blk,avg_opp_blk,avg_for_to,avg_opp_to,Win,Win_Spread,Win_Over,ML_Payout,S_Payout,O_Payout
0,ATL,1,-650,-11.0,193.5,0,-7,37.5,0.5035,0.3970,...,7.0,7.0,14.5,13.5,1,0,1,15.384615,-100,91
1,ATL,0,315,8.0,202.0,1,3,36.5,0.4600,0.4160,...,5.5,7.0,12.5,18.5,0,1,0,-100.000000,91,-100
2,ATL,0,105,1.5,194.5,1,1,40.5,0.4375,0.4720,...,4.0,8.0,16.0,19.0,0,0,1,-100.000000,-100,91
3,ATL,1,-290,-6.5,194.5,1,-2,38.0,0.4345,0.4855,...,4.0,6.5,13.5,17.0,1,1,1,34.482759,91,91
4,ATL,0,-130,-2.0,197.5,2,5,30.0,0.3935,0.4550,...,4.0,4.0,11.5,15.0,1,1,0,76.923077,91,-100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9559,WAS,0,325,8.5,216.5,42,71,37.5,0.4925,0.5180,...,2.5,2.0,16.5,13.0,0,0,1,-100.000000,-100,91
9560,WAS,0,210,6.0,220.5,42,55,42.5,0.5140,0.5285,...,4.0,3.0,16.0,12.5,0,1,1,-100.000000,91,91
9561,WAS,1,-850,-10.5,210.0,42,51,41.0,0.4510,0.5130,...,5.5,4.0,13.5,17.0,0,0,0,-100.000000,-100,-100
9562,WAS,1,-350,-7.5,204.5,42,45,38.0,0.4480,0.4300,...,3.5,3.0,16.0,17.0,1,1,1,28.571429,91,91


In [10]:
cols = [0]
final_df.drop(final_df.columns[cols],axis=1,inplace=True)
final_df.to_csv('model_data_set.csv')
final_df

,home,ML,Spread,Over,season_wins,net_points,avg_for_fg,avg_for_fgp,avg_opp_fgp,avg_for_3p,...,avg_for_blk,avg_opp_blk,avg_for_to,avg_opp_to,Win,Win_Spread,Win_Over,ML_Payout,S_Payout,O_Payout
0,1,-650,-11.0,193.5,0,-7,37.5,0.5035,0.3970,10.0,...,7.0,7.0,14.5,13.5,1,0,1,15.384615,-100,91
1,0,315,8.0,202.0,1,3,36.5,0.4600,0.4160,7.5,...,5.5,7.0,12.5,18.5,0,1,0,-100.000000,91,-100
2,0,105,1.5,194.5,1,1,40.5,0.4375,0.4720,10.5,...,4.0,8.0,16.0,19.0,0,0,1,-100.000000,-100,91
3,1,-290,-6.5,194.5,1,-2,38.0,0.4345,0.4855,11.0,...,4.0,6.5,13.5,17.0,1,1,1,34.482759,91,91
4,0,-130,-2.0,197.5,2,5,30.0,0.3935,0.4550,9.5,...,4.0,4.0,11.5,15.0,1,1,0,76.923077,91,-100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9559,0,325,8.5,216.5,42,71,37.5,0.4925,0.5180,10.0,...,2.5,2.0,16.5,13.0,0,0,1,-100.000000,-100,91
9560,0,210,6.0,220.5,42,55,42.5,0.5140,0.5285,10.5,...,4.0,3.0,16.0,12.5,0,1,1,-100.000000,91,91
9561,1,-850,-10.5,210.0,42,51,41.0,0.4510,0.5130,9.0,...,5.5,4.0,13.5,17.0,0,0,0,-100.000000,-100,-100
9562,1,-350,-7.5,204.5,42,45,38.0,0.4480,0.4300,12.0,...,3.5,3.0,16.0,17.0,1,1,1,28.571429,91,91


# Random Forest Model

In [11]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
from sklearn import tree

In [16]:
final_df = pd.read_csv('model_data_set.csv')
cols = [0]
final_df.drop(final_df.columns[cols],axis=1,inplace=True)
#final_df

In [12]:
target = final_df["S_Payout"]
#target = final_df["Win_Spread"]
# target = final_df["Win"]
#target_names = ["loss", "win"]

In [13]:
data = final_df.drop(columns=['Win', 'Win_Spread', 'Win_Over','ML_Payout','S_Payout','O_Payout'],axis=1)
feature_names = data.columns

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': [230,240,250,260,270],
    'criterion': ['gini','entropy']
}

grid = GridSearchCV(RandomForestClassifier(), param_grid, verbose=True, n_jobs=-1)

grid.fit(X_train, y_train)

In [ ]:
print(grid.best_params_)

In [20]:
#win
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
#rf = make_pipeline(StandardScaler(),RandomForestClassifier(n_estimators=200,criterion='entropy'))
#rf = RandomForestClassifier(n_estimators=200,criterion='entropy')
clf = make_pipeline(StandardScaler(),LogisticRegression(max_iter=1000, tol=1e-3))
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

ValueError: Unknown label type: 'continuous'

In [ ]:
#0.7712254286909243

In [ ]:
#winspread
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
# rf = RandomForestClassifier(n_estimators=170,criterion='gini')
# rf = rf.fit(X_train, y_train)
# rf.score(X_test, y_test)
clf = make_pipeline(StandardScaler(),LogisticRegression(max_iter=1000, tol=1e-3))
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
#0.6905060644081974

In [6]:
#winover
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
# rf = RandomForestClassifier(n_estimators=260,criterion='entropy')
# rf = rf.fit(X_train, y_train)
# rf.score(X_test, y_test)
clf = make_pipeline(StandardScaler(),LogisticRegression(max_iter=1000, tol=1e-3))
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.7185278126306984

In [ ]:
#0.6917607695524886

In [ ]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

In [7]:
from joblib import dump, load
dump(clf, 'over_model.joblib') 

['over_model.joblib']

In [15]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
regr_1 = DecisionTreeRegressor(max_depth=4)

regr_2 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=4),
                          n_estimators=300, random_state=42)

In [16]:
regr_1.fit(X_train, y_train)
regr_2.fit(X_train, y_train)

AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=4),
                  n_estimators=300, random_state=42)

In [24]:
regr_1.score(X_test, y_test)
regr_2.score(X_test, y_test)

-1.4555136050018667

In [17]:
regr_2.score(X_test, y_test)

0.09806773337146246

In [18]:
regr_1.score(X_test, y_test)

0.07890463468605657

In [19]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
regr = RandomForestRegressor(max_depth=2, random_state=42)
regr.fit(X_train, y_train)

RandomForestRegressor(max_depth=2, random_state=42)

In [20]:
regr.score(X_test, y_test)

0.08806264080179138

In [21]:
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor
regr = BaggingRegressor(base_estimator=SVR(),n_estimators=10, random_state=42)
regr.fit(X_train, y_train)

BaggingRegressor(base_estimator=SVR(), random_state=42)

In [22]:
regr.score(X_test, y_test)

-0.8701990472593992

In [23]:
from sklearn.linear_model import RidgeCV
clf = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1]).fit(X_train, y_train)

In [24]:
clf.score(X_test, y_test)

0.2257705638810089

In [25]:
from sklearn.linear_model import LassoCV
reg = LassoCV(cv=5, random_state=0).fit(X_train, y_train)
reg.score(X_test, y_test)

0.14391719236468192

In [26]:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
regr.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svr', SVR(epsilon=0.2))])

In [27]:
regr.score(X_test, y_test)

0.1700451681441243

In [28]:
from sklearn.kernel_ridge import KernelRidge
clf = KernelRidge(alpha=1.0).fit(X_train, y_train)
clf.score(X_test, y_test)

0.22604288653812854

In [33]:
from joblib import dump, load
#dump(clf, 'payout_model.joblib') 

['payout_model.joblib']

In [31]:
from sklearn import linear_model
clf = linear_model.BayesianRidge().fit(X_train, y_train)
clf.score(X_test, y_test)

0.22609237470205956

In [32]:
from joblib import dump, load
dump(clf, 's_payout_model.joblib') 

['s_payout_model.joblib']

In [30]:
from sklearn.linear_model import SGDRegressor
reg = make_pipeline(StandardScaler(),SGDRegressor(max_iter=1000, tol=1e-3)).fit(X_train, y_train)
reg.score(X_test, y_test)

0.22490080842393112